In [1]:
# loading libraries
import pandas as pd
import networkx as nx
import os
from tqdm import tqdm
import statistics
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# loading and cleaning ENSG converter
test = []
geneId_geneName = {}
with open('Homo_sapiens.GRCh37.74.gtf', 'r') as file:
    for line in file:
        line = line.strip()
        data = line.split('\t')[-1]
        test.append(data)
        if 'gene_name' in data:
            attributes = data.split(';')
            geneId = attributes[0].split(' ')[1].strip('"')
            for attr in attributes:
                if 'gene_name' in attr:
                    geneName = attr.split(' ')[2].strip('"')
                    if geneId not in geneId_geneName:
                        geneId_geneName[geneId] = geneName
geneName_geneId = {v: k for k, v in geneId_geneName.items()}

In [3]:
def createRegulatory(regulatory_filepath):
    # loading, cleaning, and permutating regulatory dataset
    print(f'Loading: {regulatory_filepath}')
    regulatory = pd.read_csv(regulatory_filepath, index_col=0)
    
    exceptions = []
    for name in regulatory.index:
        try:
            regulatory = regulatory.rename(index={name: geneName_geneId[name]})
        except:
            exceptions.append(name)
    
    print(f'Row exception count: {len(exceptions)}')
    
    for exc in exceptions:
        regulatory = regulatory.drop(exc)
    
    exceptions = []
    for ID in regulatory.columns.tolist():
        if ID not in geneId_geneName:
            exceptions.append(ID)
    
    print(f'Column exception count: {len(exceptions)}')

    for exc in exceptions:
        regulatory = regulatory.drop(exc)
    
    def inverse(x):
        return 1/x
    
    def absolute(x):
        return abs(x)
    
    regulatory = regulatory.map(inverse)
    regulatory = regulatory.map(absolute)

    return regulatory

def createEndpoints(filepath):
    # loading and cleaning dataset
    endpoints = pd.read_csv(filepath, sep='\t', index_col=0)
    
    exceptions = []
    for name in endpoints.index:
        try:
            endpoints = endpoints.rename(index={name: geneName_geneId[name.upper()]})
        except:
            exceptions.append(name)
    
    print(f'Endpoints exception count: {len(exceptions)}')
    
    for exc in exceptions:
        endpoints = endpoints.drop(exc)

    return list(endpoints.index)

def createRandomEndpoints(regulatory, num, seed):
    rand = regulatory.sample(n=num, random_state=seed)
    return list(rand.index)

def createGraph(regulatory, endpoints):
    # creating network
    regMatrix = regulatory.to_numpy().tolist()
    
    G = nx.Graph()
    nodes = list(set(list(regulatory.index) + regulatory.columns.tolist() + endpoints))
    G.add_nodes_from(nodes)
    edgeCount = 0
    for rowName, row in zip(regulatory.index, regMatrix):
        for columnName, cell in zip(regulatory.columns.tolist(), row):
            # if cell < 10:
            G.add_edge(rowName, columnName, weight=cell)
            edgeCount += 1
    
    print(f'EdgeCount: {edgeCount}')

    return G

def connectionEnrichment(origins, endpoints):
    datasets = os.listdir('data')
    count = {}
    for i, dataset in enumerate(datasets):
        print(i)
        regulatory = createRegulatory(f'data/{dataset}')
        G = createGraph(regulatory, endpoints)
        for origin in origins:
            for i, endpoint in enumerate(tqdm(endpoints)):
                try:
                    path = nx.shortest_path(G, origin, endpoint, weight="weight")
                    for node in path:
                        if node not in count:
                            count[node] = 1
                        else:
                            count[node] += 1
                except:
                    pass
    return count

def removeOrigins(dic):
    return { k:v for (k,v) in dic.items() if value > 10000}

def sortDic(dic):
    return dict(reversed(sorted(dic.items(), key=lambda item: item[1])))

def printResultsWithStats(dic):
    print(f'Size: {len(controlSortedCount2)}')
    print(f'Average: {statistics.mean(controlSortedCount2.values())}')
    print(f'Median: {statistics.median(controlSortedCount2.values())}')
    print(controlSortedCount2)

In [4]:
# defining origins
OCT4 = 'ENSG00000229094'
SOX2 = 'ENSG00000181449'
KLF4 = 'ENSG00000136826'
OSKgenes = [OCT4, SOX2, KLF4]

In [ ]:
# multi-dataset aging gene enrichment
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
agingCount = connectionEnrichment(OSKgenes, globalAgingGenes)

Endpoints exception count: 15
0
Loading: data/Testis.csv
Row exception count: 0
Column exception count: 0
EdgeCount: 19476492


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 310/310 [02:44<00:00,  1.88it/s]


1
Loading: data/Brain_Cerebellum.csv
Row exception count: 0
Column exception count: 0
EdgeCount: 19476492


  4%|████▉                                                                                                                                     | 11/310 [00:05<02:49,  1.77it/s]

In [ ]:
# removing OSKgenes from path
agingCount.pop('ENSG00000229094')
agingCount.pop('ENSG00000136826')
agingCount.pop('ENSG00000181449')

# printing results
agingSortedCount = sortDic(agingCount)
printResultsWithStats(agingSortedCount)

In [ ]:
# control experiment 1
datasets = os.listdir('data')
regulatory = createRegulatory(f'data/{datasets[0]}')
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
randomGenes = createRandomEndpoints(regulatory, len(globalAgingGenes) + len(OSKgenes), 42)
origins = randomGenes[len(globalAgingGenes):]
endpoints = randomGenes[:len(globalAgingGenes)]
controlCount1 = connectionEnrichment(origins, endpoints)

In [ ]:
# removing random 'OSKgenes' from path
control1 = removeOrigins(controlCount1)

# printing results
controlSortedCount1 = sortDic(control1)
printResultsWithStats(controlSortedCount1)

In [ ]:
# control experiment 2
datasets = os.listdir('data')
regulatory = createRegulatory(f'data/{datasets[0]}')
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
randomGenes = createRandomEndpoints(regulatory, len(globalAgingGenes) + len(OSKgenes), 43)
origins = randomGenes[len(globalAgingGenes):]
endpoints = randomGenes[:len(globalAgingGenes)]
controlCount2 = connectionEnrichment(origins, endpoints)

# removing random 'OSKgenes' from path
controlCount2.pop('ENSG00000229094')
controlCount2.pop('ENSG00000136826')
controlCount2.pop('ENSG00000181449')

In [ ]:
# removing random 'OSKgenes' from path
control2 = removeOrigins(controlCount2)

# printing results
controlSortedCount2 = sortDic(control2)
printResultsWithStats(controlSortedCount2)

In [ ]:
# control experiment 3
datasets = os.listdir('data')
regulatory = createRegulatory(f'data/{datasets[0]}')
globalAgingGenes = createEndpoints('global_aging_genes.tsv')
randomGenes = createRandomEndpoints(regulatory, len(globalAgingGenes) + len(OSKgenes), 44)
origins = randomGenes[len(globalAgingGenes):]
endpoints = randomGenes[:len(globalAgingGenes)]
controlCount3 = connectionEnrichment(origins, endpoints)

# removing random 'OSKgenes' from path
controlCount3.pop('ENSG00000229094')
controlCount3.pop('ENSG00000136826')
controlCount3.pop('ENSG00000181449')

In [ ]:
# removing random 'OSKgenes' from path
control3 = removeOrigins(controlCount3)

# printing results
controlSortedCount3 = sortDic(control3)
printResultsWithStats(controlSortedCount3)

In [ ]:
# creating graph
agingCount = [values for key, values in agingSortedDict.items()]
controlCount1 = [values for key, values in controlSortedDict1.items()]
controlCount2 = [values for key, values in controlSortedDict2.items()]
controlCount3 = [values for key, values in controlSortedDict3.items()]
agingName = ['aging' for i in range(len(agingCount))]
controlName1 = ['control1' for i in range(len(controlCount1))]
controlName2 = ['control2' for i in range(len(controlCount2))]
controlName3 = ['control3' for i in range(len(controlCount3))]

counts = agingCount + controlCount1 + controlCount2 + controlCount3
names = agingName + controlName1 + controlName2 + controlName3

df = pd.DataFrame({'counts': counts, 'names': names})

sns.set_theme(style="ticks")

f, ax = plt.subplots(figsize=(7, 5))
sns.despine(f)

sns.histplot(
    df,
    x="counts", hue="names",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())